# Demo: Run an esmvaltool recipe using containers

In this example, we demonstrate how to execute an ESMValtool recipe using containers on Spiders infrastructure.

## Prerequest: Starting a Jupyter Server on Spider.

A Jupyter Server should be started on Spider as the environment to execute this notebook. You can follow [this instruction](https://github.com/RS-DAT/JupyterDaskOnSLURM) to start a Jupyter server on Spider.

After successfully setting up the Jupyter server, please copy this notebook to the Spider file system. Then open it from the browser on your local PC (as specified in the above instructon). Effectively, this notebook run as a Slurm job on Spider.

## Step 1: build a Singularity container

ESMValTool is provided in the form of Docker containers. However on Spiders, [it is suggested to use Singularity](https://spiderdocs.readthedocs.io/en/latest/Pages/software_on_spider.html?highlight=singularity#singularity-containers) as the container technology. Therefore we will convert the existing Docker container to a Singularity container. Please run this command on Spiders:

```sh
# Step1: build sif image (this should be done once)
singularity build esmvaltool_stable.sif docker://esmvalgroup/esmvaltool:stable
```

This will build a Singularity Image File (.sif) named `esmvaltool_stable.sif` on the Spiders file system, from DockerHub. Note that this may take ~20 minutes.

## Step 2: User configeration
One can run the following command to get the user configeration file.

In [ ]:
# Get user config file
!singularity run esmvaltool_stable.sif config get_config_user

## Step 3: Execute recipe

In [ ]:
from dask.distributed import Client

client = Client("tcp://145.136.56.209:46573")
client

-- Add here the cluster --

In [ ]:
import os

# Get the abusolte path for the working directory
cwd = os.getcwd()
f_sif = '{}/esmvaltool_stable.sif'.format(cwd)
dir_recipe = '{}/recipes'.format(cwd)

# Two recipes for two datasets
recipes = ['recipe_dataset1.yml', 'recipe_dataset2.yml']

# Set up shells commands
commands = ['singularity run {} run {}/{} --offline=False'.format(f_sif, dir_recipe, r) for r in recipes]
commands

In [ ]:
# Submit the commands
future = client.map(os.system, commands)

In [ ]:
future

Once finished, one can check the downloaded climate data files and the generated results:

In [ ]:
# Check generated results
!tree ~/esmvaltool_output/